In [1]:

#the essentials for deep_learning
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D, MaxPooling2D, MaxPooling3D

#going to use tensorflow datasets for easy access to the CIFAR-10 datasets
#could have just as easily used keras dataset instead
import tensorflow_datasets as tfds

#for visualizing of results, specifically validation accuracy
from matplotlib import pyplot as plt


In [2]:
"""
created a dataset and training_set variable that basically acts as a 
pipeline to the cifar10 training split provided by tensorflow datasets

"""

training_set = tfds.load('cifar10', split = 'train')
testing_set = tfds.load('cifar10', split = 'test')



Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incomplete24N0FK/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incomplete24N0FK/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [3]:
#doing the preprocessing



def scale_images(data):


  image = data['image']
  label = data['image']
  return (image, label)







#used the scale_image to normalize RGB values between 0 and 1

training_set = training_set.map(scale_images)
testing_set = testing_set.map(scale_images)

#for both test and train datasets, I just shuffled the order

training_set = training_set.shuffle(60000)
testing_set = testing_set.shuffle(10000)

#seperated them into batches

training_set = training_set.batch(128)
testing_set  = testing_set.batch(128)

"""
#start fetching another batch while the current batch is being processed 
#by the model. The amount of batches is determined automatically based 
"""
training_set = training_set.prefetch(tf.data.AUTOTUNE)
testing_set = testing_set.prefetch(tf.data.AUTOTUNE)



#caching the datasets so that there's no need to re-preprocess between epochs
training_set = training_set.cache()
testing_set = testing_set.cache()





TypeError: ignored

In [5]:

#making the model

model = Sequential()

"""
for the first nine layers I have a series of:
A convolution layer of 16,32,64 filters respectively, each with a 3by3 kernel
 - I chose a 3 by 3 kernel because it seemed like a general theme in google
 - I chose LeakyRelu instead of Relu so I could deal with the dying ReLU problem
A dropout layer to prevent overfitting and keep the CNN on it's toes
Finally a max pooling layer to reduce computational cost
"""
model.add(Conv2D(16, (3,3),activation =LeakyReLU(0.2), input_shape=(32,32,3)))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(32, (3,3),activation =LeakyReLU(0.2)))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, (3,3),activation =LeakyReLU(0.2)))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size = (2,2)))

"""
For the next two layers I:
Flattened and passed to a fully connected layer
The dense layer which had an activation function of leakyReLU
Another dropout layer to prevent overfitting
Finally the classification layer with 10 nuerons (for 10 classes in cifar10)
 - Used softmax as activation so it would give me probabilities for each class label

"""

model.add(Flatten())
model.add(Dense(256, activation =LeakyReLU(0.2)))
model.add(Dropout(0.2))

model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 30, 30, 16)        448       
                                                                 
 dropout_1 (Dropout)         (None, 30, 30, 16)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 15, 15, 16)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 13, 13, 32)        4640      
                                                                 
 dropout_2 (Dropout)         (None, 13, 13, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                  

In [6]:
"""
Used sparse_categorical crossentrophy for the loss function because I needed
a probabilistic loss function and my options were categorical, binary, or
sparse_categorical. I ruled out binary because there were more than two classes,
and since the cifar dataset comes with integer labels, sparse_categorical makes 
more sense.


I used ada as my optimizer, while I believe I should have SGD instead, because 
adam performs well on the training set, but generalizes worse
then SGD on the testing set, however adam is faster and it has an adaptive 
learning rate, so that's what I'm going to be using
"""

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', \
              metrics = ['accuracy'])



In [7]:
history = model.fit(training_set, validation_data=testing_set, epochs = 10)

Epoch 1/10


ValueError: ignored